In [ ]:
import pandas as pd
import seaborn as sns
import requests
from datetime import datetime
import yaml
import csv
from dataclasses import dataclass, field
import uuid 
import collections

In [ ]:
pd.options.display.float_format = '{:,.2f}'.format

balance_file = 'data/account_records.csv'

accounts_file = 'data/accounts.csv'

def get_exchange_rates(base, date=None, return_currencies=None):
    
    params={'base':base}
    
    if date == None:
        url = f'https://api.exchangeratesapi.io/latest'
    else:
        url = f'https://api.exchangeratesapi.io/{date}'
        
    if return_currencies != None:
        params['symbols'] = ','.join(return_currencies)

    # Making our request
    response = requests.get(url, params=params)
    exchange_rates = response.json()
    
    return exchange_rates

def convert_currency(row, base):
        
    if row['currency'] == base:
        
        return {f'balance_{base}': row['balance'],
                'FOREX':1}
    
    else:    
        exchange_rates = get_exchange_rates(
            base='USD', 
            date=row['date'].date(), 
            return_currencies=[row['currency']])

        return {f'balance_{base}': (row['balance'] / exchange_rates['rates'][row['currency']]),
                'FOREX':exchange_rates['rates'][row['currency']]}

base = 'USD'

balance_history = pd.read_csv(balance_file, parse_dates=['date'])

balance_history[[f'balance_{base}', 'FOREX']] = balance_history.apply(
    convert_currency, 
    axis=1, 
    result_type='expand',
    args=(base,)
)

balance_history

latest_balances = balance_history.drop_duplicates(subset='account', keep='last')
latest_balances

latest_balances['balance_USD'].sum()

sns.lineplot(
    data=balance_history,
    x='date',
    y='balance_USD',
    hue='account',
    marker='o'
)

pd.read_csv(accounts_file)

In [ ]:
@dataclass
class Account():
    '''Individual bank account, investment fund, etc.'''
         
    name: str
    type: str
    country_code: str
    id: str = field(default_factory=lambda: str(uuid.uuid4())[:8])
    

In [ ]:
@dataclass
class AccountRecord():
    '''Individual record for a balance at a point in time for an account'''
    
    datetime: datetime
    account_id: str
    balance: float
    currency: str
    id: str = field(default_factory=lambda: str(uuid.uuid4())[:8])
    

In [ ]:
class UserList(collections.abc.MutableSequence):
    
    def __init__(self):
        self._inner_list = list()
        
    def __repr__(self):      
        return repr(self.to_pandas())

    def _repr_html_(self):
        return self.to_pandas().to_html()
    
    def __len__(self):
        return len(self._inner_list)

    def __delitem__(self, index):
        self._inner_list.__delitem__(index)

    def insert(self, index, value):
        self._inner_list.insert(index, value)

    def __setitem__(self, index, value):
        self._inner_list.__setitem__(index, value)

    def __getitem__(self, index):
        return self._inner_list.__getitem__(index)

    def append(self, value):
        self.insert(len(self) + 1, value)
        
    def to_pandas(self):
        
        df = pd.DataFrame([vars(x) for x in self._inner_list])
        
        if 'datetime' in df.columns:
            df['datetime'] = pd.to_datetime(df['datetime'])
        
        return df
    
    def retrieve(self, ids):
        '''Retrieves based on the unique id of the object
        
        Args:
            ids (list of str): unique id of object
            
        Returns:
            UserList: list of objects corresponding to ids
        '''
        filtered_list = list(filter(lambda item: item.id in ids, self._inner_list))
        res = type(self)()
        
        for item in filtered_list:
            res.append(item)
        
        return res
    

In [ ]:
class AccountRecordList(UserList):
    '''List of AccountRecords'''
    
    def getMostRecentRecord(self, account_ids, end_time=datetime.now()):
        '''Retrieves the value (based on the newest AccountRecord) for the specified account_id
        
        Args:
            account_ids (list of str): unique ids for the accounts
            end_time (datetime): defaults to now, but can specify time in past to get 
                balance on that date (inclusive)
        
        Returns:
            (AccountRecordList): list of most recent account records for account_ids specified
        '''
                
        records = self.to_pandas()
        filtered_records = records[(records['account_id'].isin(account_ids)) & (records['datetime'] <= end_time)]        
        idx = filtered_records.groupby(['account_id'])['datetime'].transform(max) == filtered_records['datetime']
        grouped_records = filtered_records[idx]        
        
        return self.retrieve(ids=list(grouped_records['id']))
    
    def getAccountRecords(self, account_ids):
        '''Retrieves the account records for the account_ids specified
        
        Args: 
            account_ids (str or list of str): unique id(s) for the accounts
            
        Returns:
            (AccountRecordList): list of records corresponding to account ids specified
        '''
        
        if type(account_ids) == str:
            acccount_ids = [account_ids]
        
        filtered_records = filter(lambda account_record: account_record.account_id in account_ids, self)
        
        res = AccountRecordList()
        
        res._inner_list = list(filtered_records)
        
        return res
        

In [ ]:
class AccountList(UserList):
    '''List of Accounts'''

In [ ]:
class AccountValueError(Exception):
    """Exception raised when the account is not found in the client accounts.

    Attributes:
        account (str): name of the account
        message -- explanation of the error
    """

    def __init__(self, account_id, message=f'Account was not found in list of accounts'):
        self.account_id = account_id
        self.message = message
        super().__init__(self.message)

    def __str__(self):
        return f'{self.account_id} -> {self.message}'
    

In [ ]:
class Client():
    '''represents an instance of use of the finance tool'''
    
    def __init__(self, config, accounts, account_records, 
                 country_codes, currency_codes):
        '''Initializes instance of the finance tool with static config,
        static data, and account records
        
        Args:
            config (str): path to yaml file with general config information
            accounts (str): path to csv file with account information
            accounts_records (str): path to csv file with historical account information
            country_codes (str): path to csv file with country codes
            currency_codes (str): path to csv file with currency codes
        '''
        
        self._config_file = config
        self._accounts_file = accounts
        self._accounts_records_file = account_records
        self._country_codes_file = country_codes
        self._currency_codes_file = currency_codes

        with open(config) as file:
            config = yaml.load(file, Loader=yaml.FullLoader)    
            self._account_types = config['account_types']
        
        self.account_records = AccountRecordList()
        
        with open(self._accounts_records_file, mode='r') as csv_file:
            csv_reader = csv.DictReader(csv_file)
            line_count = 0
            for row in csv_reader:
                self.account_records.append(AccountRecord(**row))

        self.accounts = AccountList()

        with open(self._accounts_file, mode='r') as csv_file:
            csv_reader = csv.DictReader(csv_file)
            line_count = 0
            for row in csv_reader:
                self.accounts.append(Account(**row))      
        
        self._country_codes = []

        with open(self._country_codes_file, mode='r') as csv_file:
            csv_reader = csv.DictReader(csv_file)
            line_count = 0
            for row in csv_reader:
                self._country_codes.append(row['Alpha-3 code'])

        
        self._currency_codes = []

        with open(self._currency_codes_file, mode='r') as csv_file:
            csv_reader = csv.DictReader(csv_file)
            line_count = 0
            for row in csv_reader:
                self._currency_codes.append(row['alpha_code'])
                    
    def addAccount(self, account):
        '''Add a new account to the client
        
        Args:
            account (Account): banking, invetment or other account
            
        '''
        
        if account.type not in self._account_types:
            raise ValueError(f'Account type: {account.type} is not valid. Options are: {self._account_types}')
            
        if account.country_code not in self._country_codes:
            raise ValueError(f'Country code: {account.country_code} is not valid. Options are: {self._country_codes}')
            
        self.accounts.append(account)
        
    def addRecord(self, account_record):
        '''Add a new Account Record to the client
        
        Args:
            account_record (AccountRecord): balance on a given date for a given account
        '''
        
        account_ids = []
        
        for account in self.accounts:
            account_ids.append(account.id)
        
        if account_record.account_id not in account_ids:
            raise AccountValueError(account_record.account_id)
            
        if account_record.currency not in self._currency_codes:
            raise ValueError(f'Currency: {account_record.currency} is not valid. Valid currencies are: {self._currency_codes}')
        
        self.account_records.append(account_record)
        

In [ ]:
c = Client(
    config='config.yml', 
    accounts='data/accounts.csv', 
    account_records='data/account_records.csv',
    country_codes='static/country_codes.csv',
    currency_codes='static/currency_codes.csv'
)

In [ ]:
c.account_records.getAccountRecords(account_ids='6c0037c1')

## Dash app

In [ ]:
import plotly.express as px
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import dash_bootstrap_components as dbc
import dash_table
from dash.exceptions import PreventUpdate

In [ ]:
# Build App
app = JupyterDash(__name__)

In [ ]:
account_dropdown = dcc.Dropdown(
    id='account_dropdown',
    options=[{'label':account.name, 'value': account.id} for account in c.accounts],
    multi=True,
    placeholder='Select account(s)'
)

In [ ]:
account_ids = ['6c0037c1']
records = c.account_records.getAccountRecords(account_ids=account_ids).to_pandas()

record_table = dash_table.DataTable(id='record_table')

In [ ]:
app.layout = html.Div([
    account_dropdown,
    record_table
])

In [ ]:
@app.callback(
    [Output('record_table', 'columns'),
     Output('record_table', 'data')],
    [Input('account_dropdown', 'value')]
)
def select_records(selected_accounts):
    if selected_accounts == None:
        raise PreventUpdate()
        
    raw_records = c.account_records.getAccountRecords(account_ids=selected_accounts).to_pandas()
    raw_accounts = c.accounts.to_pandas()
    
    merged_df = raw_records.merge(raw_accounts, left_on='account_id', right_on='id', how='left')
    cleaned_df = merged_df[['datetime','name', 'balance', 'currency']]
    
    columns = [{"name": i, "id": i} for i in cleaned_df]
    
    return columns, cleaned_df.to_dict('records')

In [ ]:
# Run app and display result inline in the notebook
app.run_server(mode='inline')

In [ ]:
c.account_records

In [ ]:
type(ac) != list

In [ ]:
import os
from os import path
import csv
from pathlib import Path

In [ ]:
path.exists('data/')

In [ ]:
data_directory_path = 'data/'

if not path.exists(data_directory_path):
    os.mkdir(data_directory_path)

In [ ]:
path.exists(data_directory_path)

In [ ]:
record_file = 'data/accounts.csv'

l = []

with open(record_file, mode='r') as csv_file:
    if csv_file.readable():
        csv_reader = csv.DictReader(csv_file)
        line_count = 0
        for row in csv_reader:
            l.append(AccountRecord(**row))

In [ ]:
l

In [ ]:
csv_file.readable()

In [ ]:
from finance.client import Client
from pathlib import Path
from dataclasses import dataclass, field
import uuid
from finance.client.data_classes.accounts import Account

In [ ]:
c = Client()

In [ ]:
c.

## Write-back testing

In [ ]:
from finance.client import Client
from finance.client.data_classes.accounts import Account

In [ ]:
c = Client()

In [ ]:
c.addAccount(Account('Test Acct', 'USA'))

In [ ]:
c.addAccount()

In [ ]:
c.writeBack()

## Database testing

In [ ]:
import sqlite3

In [ ]:
db_path = 'data/db.sqlite3'
con = sqlite3.connect(db_path)

In [ ]:
cur.execute("SELECT name FROM sqlite_master WHERE type='table'")

In [ ]:
cur = con.cursor()

In [ ]:
delete_table = 'DROP TABLE labels'
cur.execute(delete_table)

In [ ]:
cur.close()

In [ ]:
import pandas as pd

In [ ]:
cur = con.cursor()
labels_sql = '''
CREATE TABLE labels (
    id text PRIMARY KEY,
    name text,
    description text,
)
'''
cur.execute(labels_sql)

In [ ]:
cur = con.cursor()
accounts_sql = '''
CREATE TABLE accounts (
    id text PRIMARY KEY,
    name text,
    country_code text,
    label_id text,
    FOREIGN KEY (label_id) REFERENCES labels (id)
)
'''
cur.execute(accounts_sql)

In [ ]:
cur = con.cursor()
account_records_sql = '''
CREATE TABLE account_records (
    id text PRIMARY KEY,
    date text NOT NULL,
    account_id text NOT NULL,
    balance real NOT NULL,
    currency text NOT NULL,
    FOREIGN KEY (account_id) REFERENCES accounts (id)
)
'''
cur.execute(account_records_sql)

In [ ]:
cur.execute("SELECT name FROM sqlite_master WHERE type='table'")
cur.fetchall()

In [ ]:
cur = con.cursor()

accounts_sql = "INSERT INTO accounts (name, country_code, label_id, id) VALUES (?, ?, ?, ?)"

for a in c.accounts:
    cur.execute(accounts_sql, (a.name, a.country_code, a.label_id, a.id))

In [ ]:
cur = con.cursor()

account_records_sql = "INSERT INTO account_records (id, date, account_id, balance, currency) VALUES (?, ?, ?, ?, ?)"

for ar in c.account_records:
    cur.execute(
        account_records_sql, 
        (ar.id, ar.datetime, ar.account_id, ar.balance, ar.currency)
    )
    
    

In [ ]:
from finance.client.data_classes.labels import Label, LabelList
import csv

with open('data/labels.csv', mode='r') as csv_file:
    if csv_file.readable():
        csv_reader = csv.DictReader(csv_file)
        for row in csv_reader:
            LabelList.append(Label(**row)) 

In [ ]:
cur = con.cursor()

labels_sql = "INSERT INTO labels (id, name, description) VALUES (?, ?, ?)"

for l in c.labels:
    cur.execute(
        labels_sql, 
        (l.id, l.name, l.description)
    )
    
    

In [ ]:
con.close()

In [ ]:
import pandas as pd

In [ ]:
from pathlib import Path

In [ ]:
db_path=Path('data/db.sqlite3')

In [ ]:
db_path = 'data/db.sqlite3'
con = sqlite3.connect(db_path)


In [ ]:
cur.

In [ ]:
con = sqlite3.connect(db_path)
cur.execute("SELECT name FROM sqlite_master WHERE type='table'")
l = cur.fetchall()

In [ ]:
import sqlite3

con = sqlite3.connect(db_path)
con.row_factory = sqlite3.Row
cur = con.cursor()
cur.execute("SELECT * FROM accounts")
rows = cur.fetchall()

cur.close()
con.close()

for row in rows:
    values = list(row)
    keys = row.keys()
    
    row_dict = dict(zip(keys,values))
    


In [ ]:
row_dict

In [ ]:
con.commit()

In [ ]:
con.close()

In [ ]:
import csv
from finance.client.data_classes.accounts import AccountList
from finance.client.data_classes.accounts import Account

list_object = AccountList()

with open('data/accounts.csv', mode='r') as csv_file:
    if csv_file.readable():
        csv_reader = csv.DictReader(csv_file)
        for row in csv_reader:
            print(row)

In [ ]:
list_object

In [ ]:
def testfunc(a, b):
    print(a + b)

In [ ]:
t = ('a', 'b')

In [ ]:
testfunc(*t)

In [ ]:
from finance.client import Client

In [ ]:
c = Client()

In [ ]:
c.account_records

In [ ]:
from finance.client.data_classes.accounts import *
from finance.client.utils._database import _createTable

In [ ]:
_accounts_table_definition

In [ ]:
table_name = 'labels'

con = sqlite3.connect(db_path)
con.row_factory = sqlite3.Row
cur = con.cursor()
try:
    cur.execute(f'SELECT * FROM {table_name}')
except OperationalError:
    _createTable(self, self.)
rows = cur.fetchall()
cur.close()
con.close()

In [ ]:
con = sqlite3.connect(db_path)
cursor = con.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

In [ ]:
def setup_database()

## Testing UPSERT in SQLite

In [ ]:
import sqlite3

In [ ]:
ids = ['9ffeeab0']

In [ ]:
db_path = 'data/test_db.sqlite3'
con = sqlite3.connect(db_path)
cur = con.cursor()
if len(ids) > 1:    
    query_filter = ' OR '.join([f'id="{id}"' for id in ids])
    
else:
    query_filter = f'id = "{ids[0]}"'

print(query_filter)
cur.execute(f'SELECT * FROM accounts WHERE {query_filter}')
rows = cur.fetchall()
for row in rows:
    print(type(row))
    
     

In [ ]:
ids = [1,2,3]
query_filter = ' OR '.join([f'id={id}' for id in ids])
print(f'SELECT * FROM accounts WHERE {query_filter}')

In [ ]:
cur = con.cursor()
cur.execute('SELECT * FROM accounts')


In [ ]:
for row in rows:
    print(row)

In [ ]:
upsert = '''
INSERT OR REPLACE INTO accounts (id, name, country_code)
  VALUES('affe99d7', 'test_acct', 'AAA') 
'''

In [ ]:
cur = con.cursor()
cur.execute(upsert)

In [ ]:
con.row_factory = sqlite3.Row
cur = con.cursor()
cur.execute('SELECT * FROM accounts')
rows = cur.fetchall()
for row in rows:
    print(row)

In [ ]:
con.commit()

In [ ]:
records = [('444', 'test_acct', 'GGG', '000'), ('444', 'new_acct', 'XXXXX', None)]

In [ ]:
con = sqlite3.connect(db_path)
cur = con.cursor()
update_query = '''INSERT OR REPLACE INTO accounts (id, name, country_code, label_id) 
                  VALUES (?, ?, ?, ?)'''

cur.executemany(update_query, records)

In [ ]:
con.close()

In [ ]:
def updateMultipleRecords(db_path, table_name, values, records):
    '''INSERT OR REPLACE records in the specified table based on the parameters
    
    Args:
        db_path (Path): db_path (Path): path defining the location of the sqlite3 database
        table_name (str): name of table in the database
        values (tuple): corresponds to the columns to be updated in the database
        records (list): list of records with the same structure as 'values'
    '''
    con = sqlite3.connect(db_path)
    cur = con.cursor()
    
    val_placeholder = ','.join(['?']*len(values))
    
    update_query = f'''INSERT OR REPLACE INTO {table_name} {values} VALUES ({val_placeholder})'''

    cur.executemany(update_query, records)
    
    #con.commit()
    cur.close()
    con.close()

In [ ]:
updateMultipleRecords(db_path, 'accounts', ('id', 'name', 'country_code', 'label_id'), records)

In [ ]:
from finance.client import Client

In [ ]:
c = Client()

In [ ]:
for account in c.accounts:
    print(account)

In [ ]:
v

In [ ]:
from dataclasses import astuple, asdict, fields, 
from finance.client.data_classes.accounts import Account

In [ ]:
asdict(c.accounts[0])

In [ ]:
d.keys()

In [ ]:
fields(Account)

In [ ]:
a = Account(name='test', country_code='USA', id='qp30r9euqw')

In [ ]:
asdict(a)

In [ ]:
asdict()

In [ ]:
tuple(asdict(a).keys())

In [ ]:
data_types = {
    'accounts' : {
        'table_name' : 'accounts',
        'table_definition' : 'accounts_table_definition',
        'resource_type' : 'Account',
        'list_type': 'AccountList'
    },
    'account_records' : {
        'table_name' : 'account_records',
        'table_definition' : 'account_records_table_definition',
        'resource_type' : 'AccountRecord',
        'list_type': 'AccountRecordList'
    },
    'labels' : {
        'table_name' : 'labels',
        'table_definition' : 'labels_table_definition',
        'resource_type' : 'Label',
        'list_type': 'LabelList'
    }
}

In [ ]:
for data_type in data_types:
    print(data_type)

In [ ]:
from finance.client import Client
from finance.client.data_classes.accounts import Account

In [ ]:
c = Client(db_path='data/test_db.sqlite3')

In [ ]:
c.accounts.list()

In [ ]:
a = Account(name="update_test", country_code='USA', id='70daf02d')

In [ ]:
c.accounts.update(a)

In [ ]:
c.accounts.list()

In [ ]:
c.accounts

In [ ]:
c.writeBack()

In [ ]:
from dataclasses import astuple, asdict
import sqlite3

In [ ]:
def updateMultipleRecords(db_path, table_name, records):
    '''INSERT OR REPLACE records in the specified table based on the parameters.
    
    Args:
        db_path (Path): db_path (Path): path defining the location of the sqlite3 database
        table_name (str): name of table in the database
        records (list): list of records with the same structure as 'values'
            NOTE: all records must be of the same dataclass
    '''
    
    values = tuple(asdict(records[0]).keys()) # uses first record as indicative of all records
    
    tuple_records = [astuple(record) for record in records]
    
    con = sqlite3.connect(db_path)
    cur = con.cursor()
    
    val_placeholder = ','.join(['?']*len(values))
    
    update_query = f'''INSERT OR REPLACE INTO {table_name} {values} VALUES ({val_placeholder})'''

#     cur.executemany(update_query, tuple_records)
    
#     con.commit()
    cur.close()
    con.close()

In [ ]:
updateMultipleRecords('data/test_db.sqlite3', 'accounts', c.accounts)

In [ ]:
tuple(asdict(c.accounts[0]).keys())

In [ ]:
def list(name=None, country_code=None, label_id=None):
    '''Retrieves a list of accounts based on the criteria. 

    Args:
        name (str): name of account
        country_code (str): country code for region (e.g., USA)
        label_id (str): unique id of the label on the account
    Returns:
        (list): objects meeting filter criteria
    '''
    
    
    query_list = []
        
    if name:
        name_query = f'name = {name}'
        query_list.append(name_query)

    if country_code:
        country_code_query = f'country_code = {country_code}'
        query_list.append(country_code_query)
        
    if label_id:
        label_query = f'label_id = {label_id}'
        query_list.append(label_query)
        
    if len(query_list) > 0:
        query = ' AND '.join(query_list)
    else:
        query = ''
        
    return query
        

In [ ]:
list(country_code='USA', name='test')

In [ ]:
';'.join(l)

In [ ]:
name = 1
if name:
    print('t')

In [ ]:
try:
    l = 1 / 0
except:
    print('test')
    raise

In [10]:
from finance.client import Client
from finance.client.data_classes.accounts import Account
import sqlite3

In [23]:
c = Client(db_path = 'data/test_db.sqlite3')

In [31]:
c.accounts.list()

,name,country_code,label_id,id
0,Chase Checking,USA,0de1ed51,882b8dca
1,Chase Savings,USA,0de1ed51,605096bb
2,Marcus Savings,USA,0de1ed51,9c4b0e97
3,SpareBank Savings,NOR,0de1ed51,22d775cd
4,SpareBank BSU,NOR,dedicated,cb6a7a36
5,SpareBank Debit,NOR,0de1ed51,6c0037c1
6,Optum USA,USA,Dedicated,cc237820
7,Wealthfront Cash Account,USA,0de1ed51,463a9896
8,test_acct,AAA,None,affe99d7
9,test_acct,GGG,000,4444


In [24]:
c.records.list(account_id='22d775cd')

,date,account_id,balance,currency,id
0,2020-01-01,22d775cd,5008.0,NOK,ff2d14db
1,2020-02-01,22d775cd,10008.0,NOK,12e579e7
2,2020-03-01,22d775cd,15008.0,NOK,8703f9c7
3,2020-04-01,22d775cd,20008.0,NOK,3ed42cc3
4,2020-05-01,22d775cd,25008.0,NOK,1f39866d
5,2020-06-01,22d775cd,30008.0,NOK,dece3ba1
6,2020-07-01,22d775cd,235008.0,NOK,1314a58e


In [25]:
db_path = 'data/test_db.sqlite3'
table_name = 'accounts'
ids = ['a3266238']

In [32]:
def deleteMultipleRecords(db_path, table_name, ids):
    '''DELETE records in the specified table based on the parameters.
    
    Args:
        db_path (Path): db_path (Path): path defining the location of the sqlite3 database
        table_name (str): name of table in the database
        ids (str OR list of str): ids of the records to delete
    '''
    
    if type(ids) != list:
        ids = [ids]
    
    con = sqlite3.connect(db_path)
    cur = con.cursor()
        
    delete_query = f'''DELETE FROM {table_name} WHERE id=?'''
    
    try:
        for id in ids:
            cur.execute(delete_query, (id,))
            
    except:
        cur.close()
        con.close()
        raise
    
    con.commit()
    cur.close()
    con.close()

In [30]:
deleteMultipleRecords(db_path=db_path, table_name=table_name, ids=['1234'])